In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from html_analysis import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 1.95 s
Wall time: 2.7 s


In [5]:

files_list = sorted([fn for fn in os.listdir(cu.SAVES_HTML_FOLDER) if fn.endswith('.html')])
rows_list = []
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    file_name = cu.clean_text(file_name)
    with open(file_path, 'r', encoding=s.encoding_type) as f:
        html_str = f.read()
        hashed_str = cu.convert_str_to_hash(html_str)
        cypher_str = f'''MERGE (fn:FileNames {{file_name: "{file_name}"}}) SET
            fn.hashed_str = "{hashed_str}"
            RETURN fn.file_name_id;'''
        # print(cypher_str)
        with cu.driver.session() as session:
            row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
            file_name_id = row_objs_list[0]['fn.file_name_id']
            row_dict = {}
            row_dict['file_name_id'] = file_name_id
            row_dict['file_name'] = file_name
            row_dict['hashed_str'] = hashed_str
            rows_list.append(row_dict)
df = pd.DataFrame(rows_list)

In [18]:

mask_series = df.duplicated(subset='hashed_str', keep=False)
file_names_list = df[mask_series].file_name.tolist()
filenames_str = '", "'.join(file_names_list)
cypher_str = f'''MATCH (fn:FileNames)-[r]-()
    WHERE fn.file_name IN ["{filenames_str}"]
    RETURN fn'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)

In [19]:

len(row_objs_list)

306

In [21]:

cypher_str = f'''MATCH (fn:FileNames)
    WHERE fn.file_name IN ["{filenames_str}"]
    RETURN fn'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
for row_obj in row_objs_list:
    print(row_obj)

{'fn': <Node id=17172 labels=frozenset({'FileNames'}) properties={'hashed_str': '6a93a6105b740bb30b2a32f594830a8f', 'file_name': '20220520170852043704_Lead_Data_Scientist_Work_From_Home_Remote_Indeed_com.html'}>}
{'fn': <Node id=17183 labels=frozenset({'FileNames'}) properties={'hashed_str': '6a93a6105b740bb30b2a32f594830a8f', 'file_name': '20220520170929407487_Lead_Data_Scientist_Work_From_Home_Remote_Indeed_com.html'}>}
{'fn': <Node id=17187 labels=frozenset({'FileNames'}) properties={'hashed_str': '6a93a6105b740bb30b2a32f594830a8f', 'file_name': '20220520141207556219_Lead_Data_Scientist_Work_From_Home_Remote_Indeed_com.html'}>}
{'fn': <Node id=281587 labels=frozenset({'FileNames'}) properties={'hashed_str': '1906bdc61fd13b4cc2b4d4c3077cceb7', 'file_name': 'Senior_Data_Scientist_(Remote-friendly)_11245f91d4ae122b.html', 'file_name_id': 378}>}
{'fn': <Node id=281588 labels=frozenset({'FileNames'}) properties={'hashed_str': '1906bdc61fd13b4cc2b4d4c3077cceb7', 'file_name': 'Senior_Data_

In [20]:

mask_series = df.duplicated(subset='hashed_str', keep='first')
for row_index, row_series in df[mask_series].iterrows():
    file_name = row_series.file_name
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    os.remove(file_path)